<a href="https://colab.research.google.com/github/ArtemChemist/CFU_counter/blob/main/YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.1/699.1 kB 7.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt

import os
import glob
from datetime import datetime
import time
import cv2
import shutil

from ultralytics import YOLO

In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


# Import custom module with helper functions
import sys
sys.path.append('/content/drive/My Drive')
import funcs_yolo as f

Mounted at /content/drive


In [4]:
# Unzip archives with images
!unzip -q "/content/drive/My Drive/raw.zip" -d "/content"

In [5]:
p = os.path.abspath('.')
input_dir = os.path.join(p, 'raw', 'images')
X_files, binary_labels = f.read_names(input_dir)

In [6]:
X_train, X_val, y_train, y_val = f.custom_train_val_split(X_files, binary_labels, 0.2, 42 )

Full dataset: 96 images with bacteria and 481 without
Shuffling validation dataset ....
(115,) - (115,)
Shuffling trainig dataset ....
Validation X: (115,); y: (115,)
Training X: (462,); y: (462,)


In [7]:
train_img_path = os.path.join(p, 'drive', 'MyDrive', 'YOLO', 'data', 'images','train')
os.makedirs(train_img_path)
val_img_path = os.path.join(p, 'drive', 'MyDrive', 'YOLO', 'data', 'images','val')
os.makedirs(val_img_path)
train_lbl_path = os.path.join(p, 'drive', 'MyDrive', 'YOLO', 'data', 'labels','train')
os.makedirs(train_lbl_path)
val_lbl_path = os.path.join(p, 'drive', 'MyDrive', 'YOLO', 'data', 'labels','val')
os.makedirs(val_lbl_path)

In [8]:
for train_file in X_train:
  img_curr_path = os.path.join(p, 'raw', 'images', train_file)
  img_new_path = os.path.join(train_img_path, train_file)
  shutil.move(img_curr_path, img_new_path)
  lbl_curr_path = os.path.join(p, 'raw', 'labels', train_file[:-3] +'txt')
  lbl_new_path = os.path.join(train_lbl_path, train_file[:-3] +'txt')
  shutil.move(lbl_curr_path, lbl_new_path)

In [9]:
for val_file in X_val:
  img_curr_path = os.path.join(p, 'raw', 'images', val_file)
  img_new_path = os.path.join(val_img_path, val_file)
  shutil.move(img_curr_path, img_new_path)
  lbl_curr_path = os.path.join(p, 'raw', 'labels', val_file[:-3] +'txt')
  lbl_new_path = os.path.join(val_lbl_path, val_file[:-3] +'txt')
  shutil.move(lbl_curr_path, lbl_new_path)

In [10]:
model = YOLO('yolov8n.pt')

100%|██████████| 6.23M/6.23M [00:00<00:00, 296MB/s]


In [11]:
result = model.train(data  = os.path.join(p, 'drive', 'MyDrive', 'YOLO', 'data.yaml'), epochs = 50)

Ultralytics YOLOv8.0.239 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/YOLO/data.yaml, epochs=50, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

100%|██████████| 755k/755k [00:00<00:00, 102MB/s]


Overriding model.yaml nc=80 with nc=9

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drive/MyDrive/YOLO/data/labels/train... 462 images, 4 backgrounds, 0 corrupt: 100%|██████████| 462/462 [00:01<00:00, 246.58it/s]


train: New cache created: /content/drive/MyDrive/YOLO/data/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

      16/50      2.21G      1.255      1.137      1.088         66        640: 100%|██████████| 29/29 [00:18<00:00,  1.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.22it/s]

                   all        115        329      0.722        0.8       0.77      0.468



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      17/50      2.31G      1.246      1.071      1.103         53        640:  83%|████████▎ | 24/29 [00:15<00:03,  1.57it/s]


KeyboardInterrupt: 

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!scp -r /content/runs /content/drive/MyDrive/YOLO